### Задача поиск похожих по эмбеддингам

Скачиваем датасет ([источник](http://study.mokoron.com/)): [положительные](https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0), [отрицательные](https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv).

или можно через ноутбук

!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv

!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

что надо сделать 
1. объединить в одну выборку
2. на основе word2vec/fasttext/glove/слоя Embedding реализовать метод поиска ближайших твитов

на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу.

3. Проверить насколько хорошо работают подходы


In [1]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
# from tqdm import tqdm_notebook
from tqdm import notebook

#### Объеденяем 2 датасета в 1 выборку и делаем легкую предобработку текста

In [2]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
df = positive.append(negative)
df

,text
0,"@first_timee хоть я и школота, но поверь, у на..."
1,"Да, все-таки он немного похож на него. Но мой ..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
3,"RT @digger2912: ""Кто то в углу сидит и погибае..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...
...,...
111918,Но не каждый хочет что то исправлять:( http://...
111919,скучаю так :-( только @taaannyaaa вправляет мо...
111920,"Вот и в школу, в говно это идти уже надо("
111921,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :..."


In [3]:
def user_delete_re_sub (pattern, repl, string):
    return re.sub(pattern, repl, string)

df.text = np.vectorize(user_delete_re_sub)('@[\w]*', '', df.text)
df.text.head(5)

0     хоть я и школота, но поверь, у нас то же само...
1    Да, все-таки он немного похож на него. Но мой ...
2          RT : Ну ты идиотка) я испугалась за тебя!!!
3    RT : "Кто то в углу сидит и погибает от голода...
4     Вот что значит страшилка :D\nНо блин,посмотре...
Name: text, dtype: object

In [4]:
df['text']=df['text'].apply(str)
df

,text
0,"хоть я и школота, но поверь, у нас то же само..."
1,"Да, все-таки он немного похож на него. Но мой ..."
2,RT : Ну ты идиотка) я испугалась за тебя!!!
3,"RT : ""Кто то в углу сидит и погибает от голода..."
4,"Вот что значит страшилка :D\nНо блин,посмотре..."
...,...
111918,Но не каждый хочет что то исправлять:( http://...
111919,"скучаю так :-( только вправляет мозги, но я в..."
111920,"Вот и в школу, в говно это идти уже надо("
111921,"RT : Тауриэль, не грусти :( *обнял*"


In [5]:
df.to_csv("text_in.txt", header=None, index=None, sep=',', mode='w', encoding='utf-8')

In [6]:
text_in = open('text_in.txt','r', encoding='utf-8')
# print(*text_in) # текст очень большой для github

#### На основе word2vec/fasttext слоя Embedding реализоваваем метод поиска ближайших твитов (glove не установился)

In [7]:
import gensim.downloader as api

In [8]:
api.info()['models'].keys()

dict_keys(['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis'])

In [9]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
from tqdm import tqdm_notebook
from tqdm import notebook

In [10]:
# загрузим предтренированные вектора и проверим работоспособность

word_vectors = api.load("glove-wiki-gigaword-100")  # загрузим предтренированные вектора слов из gensim-data
# выведим слово наиболее близкое к 'woman', 'king' и далекое от 'man'
result = word_vectors.most_similar(positive=['woman', 'king'], negative=['man'])
print("{}: {:.4f}".format(*result[0]))

queen: 0.7699


In [11]:
# определим схожесть между словами
similarity = word_vectors.similarity('woman', 'man')
print(similarity)

similarity = word_vectors.similarity('human', 'man')
print(similarity)

similarity = word_vectors.similarity('bee', 'man')
print(similarity)

0.8323495
0.5288512
0.21199903


In [12]:
# найдем top-3 самых близких слов
result = word_vectors.similar_by_word("man", topn=3)
print(result)

result = word_vectors.similar_by_word("cat", topn=3)
print(result)

result = word_vectors.similar_by_word("mouth", topn=3)
print(result)

[('woman', 0.832349419593811), ('boy', 0.7914870977401733), ('one', 0.7788748741149902)]
[('dog', 0.8798074722290039), ('rabbit', 0.7424426674842834), ('cats', 0.7323004007339478)]
[('tongue', 0.7366125583648682), ('mouths', 0.687748908996582), ('ear', 0.6811771988868713)]


In [13]:
assert True

#Small preprocess of the answers

question = None
written = False

with open("prepared_text.txt", "w") as fout:
    with open("text_in.txt", "r", encoding='utf-8') as fin:
        for line in notebook.tqdm(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

0it [00:00, ?it/s]

In [14]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [15]:
assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open("text_in.txt", "r", encoding='utf-8') as fin:
    for line in notebook.tqdm(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 100000:
            break

0it [00:00, ?it/s]

In [16]:
sentences = [i for i in sentences if len(i) > 2]

In [17]:
sentences[0]

['школотый',
 'поверь',
 'самый',
 'd',
 'общество',
 'профилировать',
 'предмет',
 'тип']

In [18]:
modelW2V = Word2Vec(sentences=sentences, size=300, window=5, min_count=1)
modelFT = FastText(sentences=sentences, size=300, min_count=1, window=5, workers=0)

In [19]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

with open("prepared_text.txt", "r") as f:
    for line in notebook.tqdm(f):
        n_w2v = 0
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        
        vector_w2v = np.zeros(300)
        vector_ft = np.zeros(300)
        for word in question:
            if word in modelW2V.wv:
                vector_w2v += modelW2V.wv[word]
                n_w2v += 1
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        
        if counter > 100000:
            break

w2v_index.build(10)
ft_index.build(10)

0it [00:00, ?it/s]

True

In [20]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5, ) # возвращаем 5 ближайших вектора 
    return [index_map[i] for i in answers]

In [21]:
TEXT = "хоть я и школота, но поверь, у нас то же самое общество профилирующий предмет типа" # берём самый первый твит

In [22]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['"Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D"\n',
 '"Надеялась, что я добавлю ее в друзья,лохушка :D http://t.co/36k6ADyASA"\n',
 '"с утра романтик, засевший глубоко среди извилин мозга, получил суровым кирпичом действительности по голове) и вас с добры"\n',
 '"Кто круче - 1,2,3,4 ?) #Suri http://t.co/jZ0INfN4tx"\n',
 '"И вот я сижу .. А он передо мной ..в трусах , а я засыпаю :D"\n']

In [23]:
get_response(TEXT, ft_index, modelFT, index_map)

['"Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D"\n',
 '"RT : По телеку показывали ""Квартал-95"". Они пригласили Чайф! =)) Наконец-то догадались, что Киркоров и Моисеев не нужны. Вообщ"\n',
 '"Надеялась, что я добавлю ее в друзья,лохушка :D http://t.co/36k6ADyASA"\n',
 '"Кто круче - 1,2,3,4 ?) #Suri http://t.co/jZ0INfN4tx"\n',
 '"И вот я сижу .. А он передо мной ..в трусах , а я засыпаю :D"\n']

#### Выводы:
- Подходы работают относительно неплохо, особенно первые ответы (в запросе есть "то же самое", а первые ответы соджержат "похож")
- Последующие ответы оставляют желать лучшего, хотя определённая логика прослеживается. Например, "школота" - "лохушка".